Imports i posar el path a l'origen del projecte

In [2]:
import sys
from pathlib import Path
import pandas as pd

# 🔎 Busquem la carpeta arrel del projecte: "potato-dry-matter-optics-ml"
current_path = Path().resolve()
TARGET_ROOT_NAME = "potato-dry-matter-optics-ml"

while current_path.name != TARGET_ROOT_NAME and current_path.parent != current_path:
    current_path = current_path.parent

if current_path.name != TARGET_ROOT_NAME:
    raise RuntimeError(
        f"No s'ha trobat la carpeta '{TARGET_ROOT_NAME}' pujant des del directori actual: {Path().resolve()}"
    )

project_root = current_path
sys.path.append(str(project_root))

from src.dry_matter import (
    load_model_and_scaler,
    predict_dm,
    dry_matter_quality_classification,
)

In [6]:
# Rutes al model i scaler (relatives a la root del projecte)
MODEL_PATH = project_root / "data/output/test_run_2/model_prediccio_ms.h5"
SCALER_PATH = project_root / "data/output/test_run_2/scaler_X.pkl"

# Carregar model i scaler
model, scaler = load_model_and_scaler(MODEL_PATH, SCALER_PATH)

# Definim 3 patates amb el mateix format de features (sense columnas extra)
data_patates = [
    {
        "id_mostra": 1,
        "color_promig_R": 138.91971568837624,
        "color_promig_G": 121.58969690371119,
        "color_promig_B": 91.42756529642656,
        "desviació_R": 13.97793798828688,
        "desviació_G": 11.631687736651207,
        "desviació_B": 9.612857691501834,
        "canal_NIR": 0.7142860648859746,
    },
    {
        "id_mostra": 2,
        "color_promig_R": 157.24985053036397,
        "color_promig_G": 133.50820453586056,
        "color_promig_B": 97.00165388962262,
        "desviació_R": 12.510899844049487,
        "desviació_G": 10.345700740861739,
        "desviació_B": 8.74702027487379,
        "canal_NIR": 0.7400600998179171,
    },
    {
        "id_mostra": 3,
        "color_promig_R": 146.78142627618652,
        "color_promig_G": 123.3541159419632,
        "color_promig_B": 92.35700139715321,
        "desviació_R": 13.587597343102281,
        "desviació_G": 11.358018689946597,
        "desviació_B": 9.084458024543471,
        "canal_NIR": 0.7210330262907026,
    },
]

df_patates = pd.DataFrame(data_patates)
df_patates = df_patates.set_index("id_mostra")

# Columnes de features que fa servir el model
feature_cols = [
    "color_promig_R",
    "color_promig_G",
    "color_promig_B",
    "desviació_R",
    "desviació_G",
    "desviació_B",
    "canal_NIR",
]

X = df_patates[feature_cols]

# Predir matèria seca
df_patates["MS_predit"] = predict_dm(X, scaler, model)

df_patates

,color_promig_R,color_promig_G,color_promig_B,desviació_R,desviació_G,desviació_B,canal_NIR,MS_predit
id_mostra,,,,,,,,
1,138.919716,121.589697,91.427565,13.977938,11.631688,9.612858,0.714286,19.802067
2,157.249851,133.508205,97.001654,12.510900,10.345701,8.747020,0.740060,21.196672
3,146.781426,123.354116,92.357001,13.587597,11.358019,9.084458,0.721033,20.322599


In [8]:
# Classificar la qualitat de cada patata segons la matèria seca predita
df_patates["qualitat"] = df_patates["MS_predit"].apply(dry_matter_quality_classification)

# Mostrar només info rellevant
df_patates[["MS_predit", "qualitat"]]

,MS_predit,qualitat
id_mostra,,
1,19.802067,preu rebaixat
2,21.196672,bona
3,20.322599,bona
